

## Introduction
Welcome to the project, where the fusion of sports, business, and data analytics converges into the world of cricket. 
## Table of Links
### Table
| Description | Link |
| Chosen Dataset | https://www.kaggle.com/code/akshay21kumar/ipl-auction-2023/input?select=2023+Auction |

## Table of Contents
1. Executive Summary
2. Project Dashboard
3. Background to the Project
4. Critical Elements and Essential Python Libraries for Dashboard Development
5. Applied Design Theory and Decision-Making Processes in Dashboard Development
6. Comparision of Available Tools
7. Review of Chosen Datasets 
8. Visualisation of Data with Accompanying Code
9. Reflective Evaluation
10.Conclusion


## 1. Executive Summary

This project has been an immersive journey into the convergence of business and data within the cricket sector, specifically centered around the annual Indian Premier League (IPL). This league, where ten teams compete fiercely for the coveted trophy, is marked not only by yearly battles but also by auctions that happen once in three years. These auctions offer teams the unique opportunity to rebuild their squads, strategize afresh, and prepare for the upcoming seasons. It is during this crucial time that in-depth analysis becomes imperative.

This comprehensive dashboard analysis is meticulously designed to guide the Delhi Capitals (DC) team in their player selection for the upcoming auctions, strategically aiming for their first championship win. Beyond scrutinizing past investments and results, this project aims to engage a wide audience, including cricket enthusiasts and fans.Crafted with precision, the executive summary is tailored for diverse audiences, ensuring accessibility and relevance. It's structured to align each dashboard chart with specific objectives, utilizing Python and essential libraries like Pandas, Plotly, HTML, and Dash. The dataset, sourced from Kaggle, is transparently linked to ensure credibility.

The project revolves around crucial focal points: talent acquisition, investment analysis, and the optimization of team dynamics. Key metrics such as runs scored, auction prices, wickets taken, and player performance metrics like age are rigorously scrutinized. Every visualization within the dashboard, whether it's pie charts depicting team composition or bubble charts illustrating bowling strengths, offers valuable insights into identifying potential targets.

Rigor is upheld throughout the project by subjecting analytics methodologies and tools to a comprehensive review, ensuring the validity and reliability of the conclusions drawn. The journey is encapsulated in a reflective blog, shedding light on challenges faced, lessons learned, and the evolutionary path of methodologies.

The conclusive section serves as a comprehensive summary, emphasizing crucial findings and outcomes. Referenced sources validate the methodologies applied, elevating the robustness of the presented work. Essentially, this executive summary serves as a gateway to an insightful exploration of cricket analytics, providing profound insights into player performance and strategic team-building approaches aimed at clinching the championship.


## 2. Project Dashboard
This code presented herein constitutes the foundational framework for the creation of the dashboard, encompassing key data analysis, visualization, and interactive features essential for an insightful and user-engaging interface.

In [1]:
import pandas as pd
import dash
from dash import dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go

Auction_2023 = pd.read_excel("IPLDATASET1.xlsx")
df_PlayerAnalysis = pd.read_excel('IPLDATASET1.xlsx', sheet_name='Sheet2')

# Renaming columns
Auction_2023.rename(columns={'Set No.': 'Set_No.', '2023 Set': '2023_Set', 'First Name': 'First_Name',
                             'Test caps': 'Test_cap', 'ODI caps': 'ODI_cap', 'T20 caps': 'T20_cap',
                             'Previous IPLTeam(s)': 'Previous_IPL_Team(s)', 'IPL': 'Played_in_IPL',
                             '2022 Team': '2022_Team', '2022 IPL': '2022_IPL_played'}, inplace=True)

# Concatenating the First and Last names into a new Column
Auction_2023["Player_Name"] = Auction_2023["First_Name"] + " " + Auction_2023["Surname"]

# Dropping unnecessary columns
Auction_2023.drop(columns=["First_Name", "Surname", "Set_No.", "2023_Set", "Association", "T20_cap", "ODI_cap",
                           "Test_cap", "DOB"], axis=1, inplace=True)

# Re-indexing columns
Auction_2023 = Auction_2023.reindex(columns=["Player_Name", "Country", "Reserve_Price", "Auction_Price", "TEAM",
                                             "Age", "Specialism", "Batting_Style", "Bowling_Style",
                                             "Played_in_IPL", "Previous_IPL_Team(s)", "2022_Team",
                                             "2022_IPL_played", "C/U/A"])

# Filter uncapped players
uncapped_players = Auction_2023[(Auction_2023["C/U/A"] == "Uncapped") & (Auction_2023["TEAM"] != "UnSold")]

# Bar graph for uncapped players count per team
uncapped_players_count = uncapped_players['TEAM'].value_counts().reset_index()
uncapped_players_count.columns = ['TEAM', 'Count']

fig1 = px.bar(
    uncapped_players_count, 
    x='TEAM', 
    y='Count',
    labels={'Count': 'No. of players'}, 
    title="Count of uncapped players sold",
    template='plotly_dark', 
    color='Count',  # Adding color parameter for gradient effect
    color_continuous_scale=['#caeaff', '#95c8ff', '#007acc', '#004e80']  # Custom discrete color scale in lighter shades of blue
)

fig1.update_traces(showlegend=False)  # Hide the color scale legend
fig1.update_layout(
    uniformtext_minsize=8,  # Adjust the minimum font size for text labels
    bargap=0.5 , # Adjust the gap between bars (play with the value to suit your needs)
    height=320,  # Set the height of the figure
    
    margin=dict(autoexpand=True),  # Center the plot within the allocated space
)

# Calculating mean age of players within each team
mean_age_players = Auction_2023[Auction_2023['TEAM'] != 'UnSold'].groupby('TEAM')['Age'].mean().reset_index()


# Strip plot for individual player ages within each team
fig2 = px.scatter(Auction_2023[Auction_2023['TEAM'] != 'UnSold'], x='TEAM', y='Age',
                  title='Player Ages within Each Team', labels={'Age': 'Player Age', 'TEAM': 'TEAM'},
                  template='plotly_dark', hover_data={'TEAM': True, 'Age': True, 'Player_Name': True, 'Specialism': True})

# Add mean age as red dots for each team
fig2.add_trace(go.Scatter(x=mean_age_players['TEAM'], y=mean_age_players['Age'],
                          mode='markers', marker=dict(color='red', symbol='circle', size=10),
                          name='Mean Age',
                          text=mean_age_players['Age'],  # Display player age on hover
                          hoverinfo='text'))  # Show text on hover

#filtered_df['Adjusted Bubble Size'] = df_PlayerAnalysis['Wickets Taken'] * 5

fig = px.scatter(df_PlayerAnalysis, x='Auction_Price', y='Runs Scored', size='Wickets Taken',
                 hover_name='First Name', hover_data=['Auction_Price', 'Wickets Taken'],
                 labels={'Auction_Price': 'Auction Price', 'Runs Scored': 'Runs Scored'},
                 title='Auction Price vs. Runs Scored (Bubble size represents Wickets Taken)',
                 size_max=30)
# Create the Dash app
app = dash.Dash(__name__)

# Function to filter data based on dropdown selection
def filter_data(selected_value):
    if selected_value == 'DC':
        filtered_data = Auction_2023[Auction_2023['TEAM'] == 'DC']
    elif selected_value == 'All Players':
        filtered_data = Auction_2023
    elif selected_value == 'Unsold Players':
        filtered_data = Auction_2023[Auction_2023['TEAM'] == 'UnSold']
    else:
        filtered_data = Auction_2023
    return filtered_data

# Create pie chart for distribution of 'Specialism'
def generate_pie_chart(selected_value):
    filtered_data = filter_data(selected_value)
    specialism_distribution = filtered_data['Specialism'].value_counts().reset_index()
    specialism_distribution.columns = ['Specialism', 'Count']
    pie_chart = px.pie(specialism_distribution, values='Count', names='Specialism', title='Distribution of Specialism')
    pie_chart.update_layout(
    height=320,  # Set the height of the pie chart
    
    margin=dict(autoexpand=True),  # Center the plot within the allocated space
    
    )
    return pie_chart


# Update the layout for fig2 (scatter plot)
fig2.update_layout(
    height=320,  # Set the height of the figure
    margin=dict(autoexpand=True),  # Center the plot within the allocated space
)

# Update the layout with the specified design elements
app.layout = html.Div(
    style={'backgroundColor': '#1f2630', 'color': 'white', 'height': '200vh', 'padding': '0px'},
    children=[
        html.H1("Auction Analysis of DC Team", style={'textAlign': 'center'}),
        html.Div(
            className='grid-container',
            style={'display': 'grid', 'gridTemplateColumns': '1fr', 'gridTemplateRows': '1fr', 'gap': '20px', 'marginTop': '20px'},
            children=[
                html.Div(
                    className='grid-item',
                    children=[
                        html.Div([
                            html.H2("Team Stats", style={'textAlign': 'center'}),
                            html.Div([
                html.Div("Slots Left = 5", className='stats-box', style={'border': '1px solid white', 'font-weight': 'bold', 'font-size': '18px', 'width': '200px', 'height': '50px', 'display': 'flex',  'align-items': 'center'}),
                html.Div("Purse left = 4.45 Cr", className='stats-box', style={'border': '1px solid white', 'font-weight': 'bold', 'font-size': '18px', 'width': '200px', 'height': '50px', 'display': 'flex', 'justify-content': 'center', 'align-items': 'center'}),
                html.Div("Purse used = 15 Cr", className='stats-box', style={'border': '1px solid white', 'font-weight': 'bold', 'font-size': '18px', 'width': '200px', 'height': '50px', 'display': 'flex', 'justify-content': 'center', 'align-items': 'center'}),
                html.Div("Total amount given = 19.45 Cr", className='stats-box', style={'border': '1px solid white', 'font-weight': 'bold', 'font-size': '18px', 'width': '200px', 'height': '50px', 'display': 'flex',  'align-items': 'center'})
            ], style={'display': 'flex','justifyContent': 'space-between', 'alignItems': 'center' })
        ])
                    ]),
                html.Div(
                    className='grid-item',
                    children=[
                        html.H2("Team Analysis(click options to view chart)", style={'textAlign': 'center'}),
                        html.Div([
                            html.Span(html.A("1. Uncapped players", href='#', id='uncapped', n_clicks=0, style={
                                'cursor': 'pointer', 'color': 'white', 'textDecoration': 'none', 'display': 'flex','width': '200px', 'height': '50px',
                                'border': '1px solid white', 'font-weight': 'bold','font-size': '18px', 'flex': '1','textAlign': 'center'
                            })),
                            html.Span(html.A("2. Mean age distribution", href='#', id='age_distribution', n_clicks=0, style={
                                'cursor': 'pointer', 'color': 'white', 'textDecoration': 'none', 'display': 'flex','width': '200px', 'height': '50px',
                                'border': '1px solid white', 'font-weight': 'bold','font-size': '18px', 'flex': '1', 'textAlign': 'center'
                            })),
                            html.Span(html.A("3. Player analysis", href='#', id='player_analysis', n_clicks=0, style={
                                'cursor': 'pointer', 'color': 'white', 'textDecoration': 'none', 'display': 'flex','width': '200px', 'height': '50px',
                                'border': '1px solid white', 'font-weight': 'bold','font-size': '18px', 'flex': '1', 'textAlign': 'center'
                            })),
                            html.Span(html.A("4. Specialism", href='#', id='specialism', n_clicks=0, style={
                                'cursor': 'pointer', 'color': 'white', 'textDecoration': 'none', 'display': 'flex','width': '200px', 'height': '50px',
                                'border': '1px solid white', 'font-weight': 'bold','font-size': '18px','flex': '1', 'textAlign': 'center'
                            }))
                        ], style={'display': 'flex', 'justifyContent': 'space-between', 'alignItems': 'center', 'flexWrap': 'wrap', 'color': 'white'})
                    ]
                )
            ]
        ),

        html.Div(id='graph-div', style={'grid-column': '1 / span 2', 'marginTop': '20px'})
    ]
)

# Callback to display graphs in the empty space below on message click
@app.callback(
    dash.dependencies.Output('graph-div', 'children'),
    [dash.dependencies.Input('uncapped', 'n_clicks'),
     dash.dependencies.Input('age_distribution', 'n_clicks'),
     dash.dependencies.Input('player_analysis', 'n_clicks'),
     dash.dependencies.Input('specialism', 'n_clicks')]
)
def display_graph(uncapped_clicks, age_dist_clicks, player_analysis_clicks, specialism_clicks):
    ctx = dash.callback_context
    if not ctx.triggered:
        # If no button is clicked, display the image
        return html.Div([
            html.Img(src="assets\dc_image.png", style={'opacity': '0.3', 'width': '100%', 'height': '320px'}),
        ])
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == 'uncapped':
            # Code to generate graph for uncapped players
            return dcc.Graph(id='uncapped-graph', figure=fig1)
        elif button_id == 'age_distribution':
            # Code to generate age distribution graph
            return dcc.Graph(id='age-dist-graph', figure=fig2)
        elif button_id == 'player_analysis':
            # Code to generate player analysis graph
            return dcc.Graph(id='player-analysis-graph', figure=fig)
        elif button_id == 'specialism':
            # Code to generate specialism graph
            # Code to generate specialism graph
            dropdown_pie_chart = html.Div(className='grid-item', children=[
                                          dcc.Dropdown(
                                                      id='team-dropdown',
                                                      options=[
                                                                {'label': 'DC', 'value': 'DC'},
                                                                {'label': 'All Players', 'value': 'All Players'},
                                                                {'label': 'Unsold Players', 'value': 'Unsold Players'}
                                                              ],
                                                      value='DC', 
                                                      style={'color': 'black'}# Default value
                                                      ),
            dcc.Graph(id='pie-chart', figure=generate_pie_chart('DC'))
            ])
            return dropdown_pie_chart
        else:
            return ""

# Callback to update the pie chart based on dropdown selection
@app.callback(
    dash.dependencies.Output('pie-chart', 'figure'),
    [dash.dependencies.Input('team-dropdown', 'value')]
)
def update_pie_chart(selected_value):
    return generate_pie_chart(selected_value)
# External CSS styles for grid layout
external_stylesheets = [
    'https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css'
]


# Set external stylesheets in the Dash app
app.css.append_css({
    'external_url': external_stylesheets
})



# Run the app and print the URL to the console
url = "http://127.0.0.1:8050/"
print(f"Dashboard URL: {url}")
app.run_server(debug=False, port=8050, mode='external')






C:\Users\VAIBHAVI T\AppData\Local\Temp\ipykernel_38548\2070094080.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Dashboard URL: http://127.0.0.1:8050/


c:\Users\VAIBHAVI T\anaconda33\Lib\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of ['https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css'] is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



## 3. Background to the Project

The cricket sector, particularly in tournaments like the IPL, is becoming increasingly intertwined with business and data analytics.Within this landscape, the IPL, an annual summer tournament featuring 10 competitive teams, holds immense significance. Despite the sport's fervor, three teams—RCB, PBKS, and DC—have faced a drought in clinching the IPL cup since its inception. Central to bolstering DC's prospects, my focus centers on optimizing auction strategies, utilizing the triennial IPL auctions to infuse talent and secure a competitive edge in their pursuit of the coveted cup.
 
Main Objectives:
1.Acquiring Uncapped Players: The primary goal is to identify and acquire uncapped players during the upcoming auction for DC.These players represent untapped potential and can significantly contribute to the team's success.

2.Targeting Young Talent: Emphasizing a strategic approach, the focus is on targeting younger players. Investing in youthful talent ensures long-term stability for the team, relying on their fitness and potential for extended periods.

3.Strategic Player Selection: Thorough analysis of the team's composition is imperative. Careful consideration is needed to balance the team by gauging the need for wicketkeepers, bowlers, batsmen, and all-rounders based on the existing percentage of each player type within the team.

4.Evaluation of Previous Investments: Assessing the team's usual playing eleven is crucial.Delving into the expenditures made on these players and evaluating their performance in the previous season will ascertain the effectiveness of the team's investments. This analysis aims to determine if the team's investments aligned with the players' actual on-field contributions.

This project aims to utilize data analytics to make informed decisions, strategically selecting players during auctions, 
and fostering a team composition that not only elevates DC's performance in the short term but also secures a stable foundation for long-term success.

## 4. Critical Elements and Essential Python Libraries for Dashboard Development
Upon assessing the current status of the DC team, the identification of five vacant slots within a 4.5 CR purse designated for player, highlights a pivotal phase for strategic player selection. Recognizing the significance of this juncture, my focus has revolved around  considerations: distinguishing between capped and uncapped players, evaluating player age for sustained team dynamics, and scrutinizing past  performances linked to prior team investments.

Embarking on the data analysis journey, the utilization of Pandas libraries facilitated essential data cleansing tasks extracted from the xlsx. Involved eliminating redundant columns, refining existing column structures, and ensuring data integrity by addressing NaN values. Subsequent efforts led me to opt for Plotly as the primary visualization tool, given its proficiency in crafting charts example dynamic and engaging bar. infusion of HTML styling elements added a layer of visual appeal and streamlined user interaction. Dash streamlines dashboard deployment, allowing seamless integration of Plotly charts. By embedding these visualizations within the Dash app, users gain access to enhanced functionalities like dropdown menus and interactive data hovering, providing comprehensive insights at their fingertips.

## 5. Applied Design Theory and Decision-Making Processes in Dashboard Development
Dashboard Design Process: Integration with Cole's Design Principles

1. Empathy - Understanding Team Needs:
Commencing with a thorough analysis of the Indian Premier League (IPL) team's historical performance, the design process prioritizes  the team's challenges and aspirations. This understanding serves as a foundation to address existing gaps and strategize effectively through the dashboard's design.

2. Define - Clarity in Statistical Representation:
Aligning with the 'define' stage, emphasis is placed on articulating statistical data concisely. Specific elements like runs scored, wickets taken, auction prices, and the age distribution within the team were meticulously analyzed. We aimed to understand what types of players were on the team and how their skills could be used to meet our goals.

3. Ideate - Visualization Selection for Key Objectives:
The 'ideate' phase involves the meticulous selection of visualizations that best serve the project's objectives. Utilizing bar graphs for  analysis, employing bubble charts for wicket magnitude, and leveraging pie charts for percentage distribution exemplify this phase. 

4. Prototype - Development and Placement of Visualizations:
During the 'prototype' phase, visualizations were created and embedded within the dashboard framework. Initially, all four charts were displayed on a single page. However, during the subsequent stage of development, a revised approach was adopted. The new concept introduced a sequential display, showcasing one graph at a time. This modification aimed to enable focused analysis while enhancing cognitive fit. By presenting data in a sequential manner, users can concentrate on one visualization at a time, promoting deeper analysis and improving overall usability 

5. Test and Feedback - Iterative Refinement:
The 'test and feedback' phase involves rigorous testing and soliciting feedback. Iterative cycles aid in identifying flaws and refining elements, particularly focusing on aesthetics and usability.

In summary, aligning this dashboard design process with Cole's principles ensures a methodical and user-centric approach. 

Gestalt Principles and Cognitive Fit in Dashboard Design:
The design of our dashboard is intricately entwined with Gestalt principles, which elucidate how our perceptual systems organically organize visual elements into meaningful patterns and groups. By leveraging these principles, our dashboard aspires to efficiently convey information to users without imposing cognitive strain. For instance, in our scatter and strip plots representing the mean age distribution of players, the deliberate use of color variations aligns with Gestalt principles. This color differentiation swiftly communicates the mean age of players while allowing users to discern individual player ages effortlessly.

Moreover, Kopp et al. (2018) assert the significance of cognitive fit in visualizations, emphasizing the harmonization between the visualization format and the cognitive task it demands. Aligning with this notion, our chosen visualizations prioritize cognitive fit by presenting data in formats that align conceptually with the cognitive processes users employ to derive insights. This deliberate selection of visualization types ensures that users can effectively engage with the data, facilitating efficient information extraction.

I've detailed this journey in a blog post of week 2 and week 3.
The links to the blog are:
https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=34618
https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=34621
These two weeks were pivotal, marked by a seamless blend of decision-making, active development, and iterative improvements.

## 6. Review of Available Tools
In the domain of dashboard creation tools like Python, Tableau, Power BI, Google Data Studio, and QlikView/Qlik Sense are widely acknowledged. 
Introducing Domo as an alternative to discuss provides a unique perspective, resembling Power BI in certain aspects.

As a cloud-based platform, Domo enables universal data access and collaboration, facilitating real-time updates without relying on on-premises. Domo boasts a low-code or no-code design, ensuring accessibility for users without extensive coding expertise. Its intuitive interface and -and-drop functionalities streamline dashboard creation, further enhanced by prebuilt apps and industry-specific solutions with ready-to-use. Notably, Domo efficiently manages substantial data volumes.

Adding to its advantages, Domo's scalability stands out, catering to the needs of growing enterprises and accommodating expanding data. This scalability is crucial for evolving business demands and data analytics requirements.

Despite these strengths, some users have highlighted limitations in Domo's support and documentation, which might not comprehensively cover  various scenarios, differing from the extensive support and resources available within Python's vast community. However, its cost may pose hurdles for students or smaller businesses, potentially limiting skill development within its platform, unlike the broader skill set gained through programming languages

## 7. Review of Chosen Datasets 
Data Sources:
The dataset used for this dashboard project was obtained from Kaggle, a prominent platform hosting various datasets. Kaggle serves as a repository for diverse datasets contributed by the community.

Dataset Details:
The chosen dataset pertains to the IPL (Indian Premier League) auction data for the year 2023. This dataset comprises 22 columns encompassing crucial information such as player details (First Name, Surname, Country, Association, DOB), cricket statistics (Runs Scored, Wickets Taken), playing styles (Batting Style, Bowling Style), international caps (Test, ODI, T20), IPL history (Previous IPL Teams, Auction Prices), and auction-specific details (Set Number, Reserve Price, Auction Price, Team).The dataset is stored in an Excel sheet format and contains a total of 444 rows or records. 

Data Preprocessing:
During the cleaning process, an examination for missing or handled data was conducted. It's notable that the dataset exhibited no missing or NaN (Not a Number) values, indicating a relatively clean and complete dataset.

Time-frame and Update Frequency:
The IPL auction data is updated every three years, aligning with the occurrence of the IPL auctions. Therefore, the dataset's frequency of update coincides with the triennial IPL auctions.

Relevance to Dashboard Objectives:
The selection of this dataset aligns with the primary objective of identifying prospective players for team recruitment during the upcoming auctions. Attributes like Auction Price, Runs Scored, Wickets Taken, and Player Specialism serve as critical factors aiding in the evaluation and selection of potential players for the team.The dataset's comprehensive player statistics and auction-related details facilitate the identification of suitable candidates to bolster the team roster during the subsequent auction events.

## 8. Visualisation of Data with Accompanying Code
In this dedicated section , the report will delve into the analysis of three key visual representations derived from the dashboard.

1. Bar Graph of Uncapped Players:
This visual representation delineates the allocation of uncapped players among the ten auction-involved teams. Uncapped players, representing individuals yet to be chosen by any national-level team, enter the auction with a base price of 20 lakhs. Notably, these uncapped players, often younger talents accustomed to playing in varied subcontinental conditions, carry significant potential and skill. Investing in these players at a lower cost becomes a strategic tactic, providing them with a platform to excel, potentially earning national team selections and fostering personal career growth. Simultaneously, this investment serves as a boon for the team, leveraging the competitive spirit and cricketing prowess of these players.
The graphical representation highlights DC having only one uncapped player. Considering the limited remaining budget, targeting an uncapped player in the upcoming auction could be a strategic move for DC. The symbiotic nature of this investment ensures mutual growth and success for both the players and the team.

In [2]:
#Accompanying code for Bar Graph of Uncapped Players
uncapped_players_count = uncapped_players['TEAM'].value_counts().reset_index()
uncapped_players_count.columns = ['TEAM', 'Count']

fig1 = px.bar(
    uncapped_players_count,
    x='TEAM', 
    y='Count',
    labels={'Count': 'No. of players'}, 
    title="Count of uncapped players sold",
    template='plotly_dark', 
    color='Count',  # Adding color parameter for gradient effect
    color_continuous_scale=['#caeaff', '#95c8ff', '#007acc', '#004e80']  # Custom discrete color scale in lighter shades of blue
)
fig1.update_traces(showlegend=False)  # Hide the color scale legend
fig1.update_layout(
    uniformtext_minsize=8,  # Adjust the minimum font size for text labels
    bargap=0.5 , # Adjust the gap between bars (play with the value to suit your needs)
    height=320,  # Set the height of the figure
    margin=dict(autoexpand=True),  # Center the plot within the allocated space
)
fig1

c:\Users\VAIBHAVI T\anaconda33\Lib\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of ['https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css'] is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



2.Scatter and strip Plot of Mean Age Distribution:
The Scatter and Strip Plot of Mean Age Distribution stands as the second pivotal visualization in my dashboard. This plot encompasses two distinct sets of points: blue points representing the ages of individual players within the team and red dots indicating the mean age of the team. Typically, cricket players retire around the age of 33, prompting teams to conscientiously manage and anticipate the retirement phase from the onset. Within this context, it becomes crucial for teams to ensure a blend of experienced players and younger talents for long-term stability. Notably, DC exhibits a roster with four players nearing retirement age, signaling a pressing need for younger and backup players to fortify the team's longevity. With their mean age surpassing 31, the team's composition emphasizes the necessity for rejuvenation and the infusion of youthful vigor into the squad.

Team success in cricket hinges significantly not only on skill but also on stability and fitness, aspects inherently linked to players' ages. 
The need for a balanced roster encompassing experienced leaders and promising younger players becomes imperative for sustained success. 
In light of this analysis, it becomes evident that DC's strategy for the upcoming auction should prioritize targeting young and dynamic players. 
Securing a pool of youthful talents will not only infuse vitality into the team but also ensure a robust and enduring framework for future achievements

In [3]:
# Accompanying code for mean age of players within each team (Scatter and strip plot)
mean_age_players = Auction_2023[Auction_2023['TEAM'] != 'UnSold'].groupby('TEAM')['Age'].mean().reset_index()


# Strip plot for individual player ages within each team
fig2 = px.scatter(Auction_2023[Auction_2023['TEAM'] != 'UnSold'], x='TEAM', y='Age',
                  title='Player Ages within Each Team', labels={'Age': 'Player Age', 'TEAM': 'TEAM'},
                  template='plotly_dark', hover_data={'TEAM': True, 'Age': True, 'Player_Name': True, 'Specialism': True})

# Add mean age as red dots for each team
fig2.add_trace(go.Scatter(x=mean_age_players['TEAM'], y=mean_age_players['Age'],
                          mode='markers', marker=dict(color='red', symbol='circle', size=10),
                          name='Mean Age',
                          text=mean_age_players['Age'],  # Display player age on hover
                          hoverinfo='text'))  # Show text on hover
fig2

3.Bubble Chart ( AUction Price vs Runs Scored (Bubble size represent Wickets- Taken):This Chart portraying Auction Price versus Runs Scored and Wickets Taken emerges as the third essential visualization within the dashboard. Here, the x-axis signifies the previous investments made during auctions, the y-axis represents the runs scored by the players, while the size of the bubbles correlates to the number of wickets taken by the usual playing 11 in the preceding year.

This visualization serves as a critical evaluation tool for DC to retrospectively analyze their past strategic decisions and their subsequent outcomes. Notably, the team's investments in players like Mukesh at 550 Lakhs yielded significant success, with Mukesh emerging as a -performing asset by securing 21 wickets, validating the substantial investment made in him. Similarly, the investment of 220 Lakhs in David Warner proved fruitful as he emerged as the top scorer in the season, validating the investment with an outstanding performance. Furthermore, Ishant's  acquisition at 50 Lakhs proved to be an astute move, contributing significantly to the team's success by securing 14 crucial wickets.
 From a business perspective, this analysis delineates the significance of strategic investments in cricketing talent. It underscores the correlation between investments made during auctions and the subsequent on-field performances, elucidating how prudent investments in players, considering their past performance and potential contributions, can significantly impact a team's success. The success stories of Mukesh, David Warner, and Ishant exemplify the importance of judicious financial allocations in securing high-performing players, reinforcing the team's competitiveness and augmenting its chances for success in future tournaments.

In [4]:
fig = px.scatter(df_PlayerAnalysis, x='Auction_Price', y='Runs Scored', size='Wickets Taken',
                 hover_name='First Name', hover_data=['Auction_Price', 'Wickets Taken'],
                 labels={'Auction_Price': 'Auction Price', 'Runs Scored': 'Runs Scored'},
                 title='Auction Price vs. Runs Scored (Bubble size represents Wickets Taken)',
                 size_max=30)
fig

c:\Users\VAIBHAVI T\anaconda33\Lib\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of ['https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css'] is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



## 9. Reflective Evaluation
Delving into sports analytics, especially cricket, was a natural progression given my profound interest in the sport. I aimed to bind business data with sports, recognizing its significance encompassing advertising, investments, leagues, player demands, and performance metrics.

Coming from a coding background, employing Python's Plotly for chart creation in the Dash app felt like somehow comfortable. However, incorporating HTML elements for styling components was a fresh challenge that expanded my hands-on expertise.

Obtaining the right data was an initial concern, but Kaggle emerged as a treasure trove of extensive datasets, providing the information pivotal
 for in-depth player and team analysis in the dashboard.

Throughout this project, I honed technical skills in data analysis, visualization, and informed decision-making.
 Simultaneously, it fostered the development of soft skills such as effective time management and adept problem-solving, enriching my overall skill set.

## 10. Conclusion
The BEMM461 project delved into the intricate fusion of sports, business, and data analytics, focusing on the dynamic realm of cricket,particularly within the Indian Premier League (IPL).
Employing a systematic dashboard design process aligned with established principles such as Cole's Design Principles and Gestalt principles fortified the creation of an intuitive and user-centric dashboard. 
The dataset obtained from Kaggle emerged as a cornerstone, empowering comprehensive player and auction analysis.The dashboard's key visualizations offered actionable insights, emphasizing untapped talent among uncapped players, the significance of age dynamics in team composition, and the strategic impact of auction investments on player performances.


## References
These references encompass a range of resources including platforms, datasets, articles, and comparative analyses, providing diverse  and information relevant to dashboard creation and data analysis

Kopp, T., Riekert, M., & Utz, S. (2018). When cognitive fit outweighs cognitive load: redundant data labels in charts increase accuracy and speed of information extraction. Computers in Human Behavior, 86, 367-376. https://doi.org/10.1016/j.chb.2018.04.037

TechnologyAdvice. (n.d.). Power BI vs. Domo. https://technologyadvice.com/business-intelligence/resources/power-bi-vs-domo/#:~:text=Power%20BI%20is%20a%20solid,based%20on%20real%2Dtime%20data.

Few, S. (2015). Effective dashboard design. ResearchGate. https://www.researchgate.net/publication/286996830_Effective_dashboard_design

Ni, H., & Wang, H. (2020). Interactive web-based data visualization with R, plotly, and shiny. ResearchGate. https://www.researchgate.net/publication/340147564_Interactive_Web-Based_Data_Visualization_with_R_plotly_and_shiny

ChatGPT. (n.d.). OpenAI Chat. https://chat.openai.com/

IPLT20. (n.d.). IPL Auction. https://www.iplt20.com/auction

Kaggle. (2023). 2023 Auction. https://www.kaggle.com/code/akshay21kumar/ipl-auction-2023/input?select=2023+Auction

Plotly. (n.d.). Plotly. https://plotly.com/

